In [1]:
import numpy as np
import cv2
import math
import os
from scipy import ndimage
%run segmentation.ipynb import output

True


Do the Gaussian Blur:
1. g_kernel(): construct gaussian kernel for size
2. convolution(): apply it on image

In [2]:
def convolution(image, kernel):
    image_row, image_col = image.shape
    kernel_row, kernel_col = kernel.shape
 
    mid_height = int((kernel_row - 1) / 2)
    mid_width = int((kernel_col - 1) / 2)
    conv_img = np.zeros(image.shape)
 
    mid_image = np.zeros((image_row + (2 * mid_height), image_col + (2 * mid_width)))
 
    mid_image[mid_height:mid_image.shape[0] - mid_height, mid_width:mid_image.shape[1] - mid_width] = image

    for row in range(image_row):
        for col in range(image_col):
            conv_img[row, col] = np.sum(kernel * mid_image[row:row + kernel_row, col:col + kernel_col])
            conv_img[row, col] = conv_img[row, col] / (kernel.shape[0] * kernel.shape[1])
            
    conv_img = cv2.normalize(conv_img, None, 0, 1, cv2.NORM_MINMAX)
    conv_img = cv2.normalize(conv_img, None, 0, 255, cv2.NORM_MINMAX)
    conv_img = conv_img.astype(np.uint8)
        
 
    return conv_img

In [3]:
def norm(x, mu, dev):
    return 1 / (np.sqrt(2 * np.pi) * dev) * np.e ** (-np.power((x - mu) / dev, 2) / 2)
 
def g_kernel(size, dev):
    kernel = np.linspace(-(size // 2), size // 2, size)
    for i in range(size):
        kernel[i] = norm(kernel[i], 0, dev)
    kernel_2D = np.outer(kernel.T, kernel.T)
 
    kernel_2D *= 1.0 / kernel_2D.max()
 
    return kernel_2D

In [4]:
def gaussian_blur(image, kernel_size):
    kernel = g_kernel(kernel_size, math.sqrt(kernel_size))
    return convolution(image, kernel)

Do the gradient calculation

In [5]:
def cal_gradient(img):
    height,width=img.shape

    #Calculating the gradients
    Ix = cv2.Sobel(np.float32(img), cv2.CV_64F, 1, 0, 3)
    Iy = cv2.Sobel(np.float32(img), cv2.CV_64F, 0, 1, 3)

    g, angle = cv2.cartToPolar(Ix, Iy, angleInDegrees=True)

    return g, angle

Do the non-maximum suppression:
1. for every point, find the neighbors in every angle
2. If it does not have the highest intensity, it will be 0.

In [6]:
def non_max_suppression(g, angle):
    g_max=np.max(g)
    weak=g_max*0.1
    height,width=g.shape
    
    for x in range(width):
        for y in range(height):
            
            d_angle=angle[y,x]
            d_angle = abs(d_angle-180) if abs(d_angle)>180 else abs(d_angle)
            
            # selecting the neigbours according to the gradient direction
            if d_angle <= 22.5:
                neighb1_x,neighb1_y = x-1,y
                neighb2_x,neighb2_y = x+1,y
            elif d_angle > 22.5 and d_angle <= (22.5+45):
                neighb1_x,neighb1_y = x-1,y-1
                neighb2_x,neighb2_y = x+1,y+1
            elif d_angle > (22.5+45) and d_angle <= (22.5+90):
                neighb1_x,neighb1_y = x,y-1
                neighb2_x,neighb2_y = x,y+1
            elif d_angle > (22.5+90) and d_angle <= (22.5+135):
                neighb1_x,neighb1_y = x-1,y+1
                neighb2_x,neighb2_y = x+1,y-1
            elif d_angle > (22.5+135) and d_angle <= (22.5+180):
                neighb1_x,neighb1_y = x-1,y
                neighb2_x,neighb2_y = x+1,y
            
            # Non-maximum supression step:
            # If it does not have the highest intensity, it will be 0.
            if width>neighb1_x>=0 and height>neighb1_y>=0:
                if g[y,x]<g[neighb1_y,neighb1_x]:
                    g[y,x]=0
                    continue

            if width>neighb2_x>=0 and height>neighb2_y>=0:
                if g[y,x]<g[neighb2_y,neighb2_x]:
                    g[y,x]=0

    return g, weak

Do the thredhold part:
If the intensity is lower than weak, it will be set to 0.

In [7]:
def threshold(img, weak):
    height,width=img.shape
    for x in range(width):
        for y in range(height):
            if img[y,x] < weak:
                img[y,x]=0
 
    return img

Do the Hysteresis part:
1. For every weak pixel, if its any neighbor is strong pixel, it will set to be strong pixel.
2. If not, it will be 0.
3. its neighbors are 8 pixels that around it.

In [8]:
def hysteresis(img, weak, strong=255):
    M, N = img.shape  
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                # neighbor has strong pixel
                if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                    or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                    or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                    img[i, j] = strong
                else:
                    img[i, j] = 0
    return img

Canny edge detection:
1. gaussian blur
2. cal_gradient
3. non_max_suppression
4. threshold
5. hysteresis

In [9]:
# implementing canny edge detection
def imple_canny(img):
    
    #Noise reduction step is doing gaussian_blur
    img = gaussian_blur(img, 7)
    
    g, angle = cal_gradient(img)
    
    #setting the minimum and maximum thresholds for double thresholding
    sup_output, weak = non_max_suppression(g, angle)

    thres_output = threshold(sup_output, weak)
    
    output = hysteresis(thres_output, 50)

    # change image type to uint8
    output = output.astype(np.uint8)

    return output
    

### The main part of the code.

In [10]:
def main(imgpath):
    image = cv2.imread(imgpath)
    image = cv2.resize(image, (800,600))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    canny_img = imple_canny(gray_image)
    
    # find contours
    cnts,new = cv2.findContours(canny_img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:30]
    
    screenCnt = None
    # find plate
    for c in cnts:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
        # find the rectangle, so the length is 4.
        if len(approx) == 4: 
            screenCnt = approx

            # save the height and width of the license plate
            x,y,w,h = cv2.boundingRect(c)
            plate_img=image[y:y+h,x:x+w]
            
            break
    
    result = output(plate_img)
    
    
    return plate_img, result

In [11]:
# if __name__ == '__main__':
#     imgpath = 'car_demo/1001.jpg'
#     img, result = main(imgpath)
#     print(result)

itzy
1AYA727
